# Exploratory data analysis
<input type="checkbox"> Import data <br>
<input type="checkbox"> Prepare data for further analysis <br>
<input type="checkbox"> Analyzing datasets <br>
<input type="checkbox"> Summarize the main characteristics <br>

In [2]:
import os,sys
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
sns.set()
# sns.set(style="darkgrid")
plt.style.use('ggplot')

## Import data from online source

In [16]:
data = pd.read_html("https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#",
                   header=1,index_col=0)

In [22]:
pd.DataFrame(data[0])

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
1,,,,,,
2,Hubei,Mainland China,1/30/2020 21:30,5806,204.0,116.0
3,Zhejiang,Mainland China,1/30/2020 21:30,537,NaN,9.0
4,Guangdong,Mainland China,1/30/2020 21:30,393,NaN,11.0
5,Henan,Mainland China,1/30/2020 21:30,352,2.0,3.0
6,Hunan,Mainland China,1/30/2020 21:30,332,NaN,2.0
7,Jiangxi,Mainland China,1/30/2020 21:30,240,NaN,7.0
8,Anhui,Mainland China,1/30/2020 21:30,237,NaN,3.0
9,Chongqing,Mainland China,1/30/2020 21:30,206,NaN,1.0
10,Shandong,Mainland China,1/30/2020 21:30,178,NaN,2.0
